In [1]:
%cd ..

/home/romet/projects/ut/e2e-rally-estonia


In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchvision

from dataloading.nvidia import NvidiaDataset, NvidiaTrainDataset, NvidiaValidationDataset, Normalize
from pilotnet import PilotnetControl, PilotNetConditional, PilotNetConditionalOld
from metrics.metrics import calculate_open_loop_metrics
from viz.analytics import create_waypoint_error_plot

from metrics.metrics import calculate_closed_loop_metrics, calculate_trajectory_open_loop_metrics, calculate_open_loop_metrics, read_frames_driving, read_frames_expert
from trajectory import calculate_steering_angle
import math

from trainer import ControlTrainer, ConditionalTrainer
from torchvision import transforms

from camera_frame import CameraFrameTransformer


%load_ext autoreload 
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def load_dataset(paths):
        return NvidiaDataset(paths,
                             transform=tr,
                             output_modality="steering_angle", 
                             metadata_file="nvidia_frames.csv")

In [4]:
def load_model(model_name, n_outputs=1, remove_first_bn=False):
    model = PilotNetConditional(n_branches=3, n_outputs=n_outputs)
    if remove_first_bn:
        model.features = nn.Sequential(*[mod for mod in model.features[1:]])
    model.load_state_dict(torch.load(f"models/{model_name}/best.pt"))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    return model

In [5]:
from trainer import ConditionalTrainer


def calculate_metrics(steering_predictions, frames_df):
    true_steering_angles = frames_df.steering_angle.to_numpy()
    fps=30
    metrics = calculate_open_loop_metrics(steering_predictions, true_steering_angles, fps=fps)
    
    left_turns = frames_df["turn_signal"] == 0
    left_metrics = calculate_open_loop_metrics(steering_predictions[left_turns], true_steering_angles[left_turns], fps=fps)
    metrics["left_mae"] = left_metrics["mae"]

    straight = frames_df["turn_signal"] == 1
    straight_metrics = calculate_open_loop_metrics(steering_predictions[straight], true_steering_angles[straight], fps=fps)
    metrics["straight_mae"] = straight_metrics["mae"]

    right_turns = frames_df["turn_signal"] == 2
    right_metrics = calculate_open_loop_metrics(steering_predictions[right_turns], true_steering_angles[right_turns], fps=fps)
    metrics["right_mae"] = right_metrics["mae"]
    return metrics


def calculate_steering_metrics(model, dataset):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False, num_workers=16, pin_memory=True,
                            persistent_workers=True)

    trainer = ConditionalTrainer(n_conditional_branches=3)
    steering_predictions = trainer.predict(model, dataloader)
    return calculate_metrics(steering_predictions, dataloader.dataset.frames)

def waypoints_to_steering_angle(predictions):
    pred_steering_angles = []
    wp_progress_bar = tqdm(total=len(predictions), smoothing=0)
    wp_progress_bar.set_description("Calculating steering angles")

    transformer = CameraFrameTransformer()

    for wp in predictions:
        wp_baselink = transformer.transform_waypoints(wp, "interfacea_link2")
        pred_steering_angles.append(calculate_steering_angle(wp_baselink))
        wp_progress_bar.update(1)
    return np.array(pred_steering_angles)

def calculate_wp_metrics(model, dataset):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False, num_workers=16, pin_memory=True,
                            persistent_workers=True)

    trainer = ConditionalTrainer(n_conditional_branches=3)
    wp_predictions = trainer.predict(model, dataloader)
    steering_predictions = waypoints_to_steering_angle(wp_predictions)
    true_steering_angles = dataloader.dataset.frames.steering_angle.to_numpy()
    return calculate_metrics(steering_predictions, dataloader.dataset.frames)

In [6]:
tr = transforms.Compose([Normalize()])
dataset_path = Path("/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021")
elva_dataset = NvidiaDataset([
    dataset_path / "2021-06-07-14-20-07_e2e_rec_ss6",
    dataset_path / "2021-06-07-14-06-31_e2e_rec_ss6",
    dataset_path / "2021-06-07-14-09-18_e2e_rec_ss6",
    dataset_path / "2021-06-07-14-36-16_e2e_rec_ss6",
    dataset_path / "2021-10-26-10-49-06_e2e_rec_ss20_elva",
    dataset_path / "2021-10-26-11-08-59_e2e_rec_ss20_elva_back"
], n_branches=3, metadata_file="nvidia_frames_ext.csv")

steering_1_model = load_model("20220627193827_steering-conditional-pre", remove_first_bn=True)
steering_1_metrics = calculate_steering_metrics(steering_1_model, elva_dataset)

steering_2_model = load_model("20220628234819_steering-conditional-pre-2")
steering_2_metrics = calculate_steering_metrics(steering_2_model, elva_dataset)

steering_overfit_model = load_model("20220629083303_steering-conditional-pre-overfit")
steering_overfit_metrics = calculate_steering_metrics(steering_overfit_model, elva_dataset)

#metrics_df = pd.DataFrame.from_dict(steering_1_metrics, index="Steering 1")
metrics_df = pd.DataFrame(steering_1_metrics, index=["Steering 1"])
metrics_df = metrics_df.append(pd.DataFrame(data=steering_2_metrics, index=["Steering 2"]))
metrics_df = metrics_df.append(pd.DataFrame(data=steering_overfit_metrics, index=["Steering overfit"]))
metrics_df

/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-20-07_e2e_rec_ss6: lenght=25833, filtered=4
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-06-31_e2e_rec_ss6: lenght=3002, filtered=1
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-09-18_e2e_rec_ss6: lenght=4550, filtered=2
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-36-16_e2e_rec_ss6: lenght=25351, filtered=18
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva: lenght=33034, filtered=11
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: lenght=33278, filtered=3


  0%|          | 0/1954 [00:00<?, ?it/s]

  0%|          | 0/1954 [00:00<?, ?it/s]

  0%|          | 0/1954 [00:00<?, ?it/s]

,mae,rmse,max,whiteness,expert_whiteness,left_mae,straight_mae,right_mae
Steering 1,7.695730,19.990115,387.792314,76.621284,24.682909,21.349283,5.772461,57.263860
Steering 2,7.257263,18.977927,369.189614,78.532692,24.682909,22.162610,5.478415,51.559928
Steering overfit,6.563537,15.081966,332.122459,63.799831,24.682909,20.429373,5.183029,39.387768


In [7]:
elva_dataset_wp = NvidiaDataset([
    dataset_path / "2021-06-07-14-20-07_e2e_rec_ss6",
    dataset_path / "2021-06-07-14-06-31_e2e_rec_ss6",
    dataset_path / "2021-06-07-14-09-18_e2e_rec_ss6",
    dataset_path / "2021-06-07-14-36-16_e2e_rec_ss6",
    dataset_path / "2021-10-26-10-49-06_e2e_rec_ss20_elva",
    dataset_path / "2021-10-26-11-08-59_e2e_rec_ss20_elva_back"
], n_branches=3, output_modality="waypoints", metadata_file="nvidia_frames_ext.csv")

waypoint_1_model = load_model("20220614175749_waypoints-balanced", n_outputs=20)
waypoint_1_metrics = calculate_wp_metrics(waypoint_1_model, elva_dataset_wp)
metrics_df = metrics_df.append(pd.DataFrame(data=waypoint_1_metrics, index=["Waypoints 1"]))

waypoint_2_model = load_model("20220629014408_waypoints-balanced-2", n_outputs=20)
waypoint_2_metrics = calculate_wp_metrics(waypoint_2_model, elva_dataset_wp)
metrics_df = metrics_df.append(pd.DataFrame(data=waypoint_2_metrics, index=["Waypoints 2"]))

waypoint_overfit_model = load_model("20220629124219_waypoints-balanced-overfit", n_outputs=20)
waypoint_overfit_metrics = calculate_wp_metrics(waypoint_overfit_model, elva_dataset_wp)
metrics_df = metrics_df.append(pd.DataFrame(data=waypoint_overfit_metrics, index=["Waypoints overfit"]))
metrics_df

/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-20-07_e2e_rec_ss6: lenght=25710, filtered=127
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-06-31_e2e_rec_ss6: lenght=2802, filtered=201
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-09-18_e2e_rec_ss6: lenght=4474, filtered=78
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-36-16_e2e_rec_ss6: lenght=25255, filtered=114
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva: lenght=32738, filtered=307
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-11-08-59_e2e_rec_ss20_elva_back: lenght=33000, filtered=281


  0%|          | 0/1938 [00:00<?, ?it/s]

  0%|          | 0/123979 [00:00<?, ?it/s]

  0%|          | 0/1938 [00:00<?, ?it/s]

  0%|          | 0/123979 [00:00<?, ?it/s]

  0%|          | 0/1938 [00:00<?, ?it/s]

  0%|          | 0/123979 [00:00<?, ?it/s]

,mae,rmse,max,whiteness,expert_whiteness,left_mae,straight_mae,right_mae
Steering 1,7.695730,19.990115,387.792314,76.621284,24.682909,21.349283,5.772461,57.263860
Steering 2,7.257263,18.977927,369.189614,78.532692,24.682909,22.162610,5.478415,51.559928
Steering overfit,6.563537,15.081966,332.122459,63.799831,24.682909,20.429373,5.183029,39.387768
Waypoints 1,10.821430,29.887013,664.220807,150.251544,25.003687,48.764697,7.399839,88.808497
Waypoints 2,12.012721,28.627826,563.243421,176.254806,25.003687,54.869180,8.590941,86.675552
Waypoints overfit,10.303916,23.383190,561.053920,117.071618,25.003687,52.269235,7.439137,68.688807


In [9]:
metrics_df = metrics_df[["mae", "left_mae", "straight_mae", "right_mae", "max", "whiteness"]]
metrics_df

,mae,left_mae,straight_mae,right_mae,max,whiteness
Steering 1,7.695730,21.349283,5.772461,57.263860,387.792314,76.621284
Steering 2,7.257263,22.162610,5.478415,51.559928,369.189614,78.532692
Steering overfit,6.563537,20.429373,5.183029,39.387768,332.122459,63.799831
Waypoints 1,10.821430,48.764697,7.399839,88.808497,664.220807,150.251544
Waypoints 2,12.012721,54.869180,8.590941,86.675552,563.243421,176.254806
Waypoints overfit,10.303916,52.269235,7.439137,68.688807,561.053920,117.071618


In [13]:
metrics_df[0:3].mean()

mae               7.172177
left_mae         21.313755
straight_mae      5.477969
right_mae        49.403852
max             363.034796
whiteness        72.984603
dtype: float64

In [15]:
metrics_df[3:].mean()

mae              11.046022
left_mae         51.967704
straight_mae      7.809972
right_mae        81.390952
max             596.172716
whiteness       147.859323
dtype: float64